In [1]:
import arff
from math import inf
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA
from scipy.stats import pointbiserialr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Prepare Data
### Load data

In [2]:
data_freq = arff.load('freMTPL2freq.arff')
data_sev = arff.load('freMTPL2sev.arff')

# Make them pandas databases
df_freq = pd.DataFrame(data_freq, columns=[
    "IDpol", "ClaimNb", "Exposure", "Area", "VehPower", "VehAge", "DrivAge", "BonusMalus", "VehBrand", "VehGas", "Density", "Region"])
df_sev = pd.DataFrame(data_sev, columns=["IDpol", "ClaimAmount"])

### Deal with categorical values

In [3]:
# Use OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')
obj_columns = df_freq.select_dtypes(include='object').columns
encoded_objs = encoder.fit_transform(df_freq[obj_columns])
encoded_columns = [f'{col}_{i+1}' for col in obj_columns for i in range(encoded_objs.shape[1])]
df_encoded = pd.concat(
    [
        df_freq.drop(obj_columns, axis=1),
        pd.DataFrame(encoded_objs, columns=encoder.get_feature_names_out(obj_columns))
    ],
    axis=1)
# To make sure ' would not create some problems down the line
df_encoded.columns = df_encoded.columns.str.replace("'", "")

## Other attempts included clustering and dictionaries

### Feature Engeneering

In [4]:
def divide(df, columns):
    return df[columns[0]] / df[columns[1]]

def prod(df, columns):
    return df[columns].product(axis=1)

def merge_columns(df, operation, columns):
    new_column_name = "@".join(columns)
    df[new_column_name] = operation(df, columns)
    return df
    
# Combine VehAge and VehPower
df_eng = merge_columns(df_encoded, prod, ["VehAge", "VehPower"])
## Ideally, one should try a few more. I made one to suggest how I would go about in a more invovled project.

### Combining and priming Dataframes

In [12]:
# Add the preict values whereevera available. Wherever not, make it into a separate file df_rest.
# The ClaimAmount / Exposure will be added to df_rest, as the goal of this project.
df_merged_1 = pd.merge(df_eng, df_sev, on="IDpol").drop("IDpol", axis=1)
df_rest = df_eng[~df_eng['IDpol'].isin(df_sev['IDpol'])].drop("IDpol", axis=1)

# As we are interested in ClaimAmount over time, replace ClaiAmount with this new column.
df_merged = merge_columns(df_merged_1, divide, ["ClaimAmount", "Exposure"])
df_merged = df_merged.drop("ClaimAmount", axis = 1)

# Our goal is to obtain df_rest["ClaimAmount@Exposure"].
# The other columns are there to avaluate our resultant values in this column.
# We first populate df_rest with empty columns, as to avoid potential errors later. We will fill in the values over time.
df_rest["ClaimAmount@Exposure"] = np.nan
df_rest["MSE"] = np.nan
df_rest["R-Squared"] = np.nan
df_rest["Adj.R-Squared"] = np.nan
df_rest["F-Statistic"] = np.nan
df_rest["p-Value"] = np.nan

## Create Model
### Define main preparatory functions

In [13]:
# Function to remove outliers. 
## Increase 1.5 to remove less.
def remove_outliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.05)
        Q3 = df[column].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

## This is now mainly usefd for returning "top_correlations": the features which the linear model below will use.
## It was also used initally by me to examine the dataset, see for corrolations and help determin my course of action.
def correlation_analysis(df):
    num_cols = [col for col in df.columns if "_" not in col]
    correlation_matrix = df[num_cols].corr()
    top_correlations = correlation_matrix["ClaimAmount@Exposure"].abs().drop("ClaimAmount@Exposure").sort_values(ascending=False).head(7)

    ## To visualise the top correlations.
    # print(f"Top Correlations:")
    # print(top_correlations)
    # sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    # plt.show()

    ## The columns with _ in the colum are ones coming form OneHotEncoder and are just 0's and 1'.
    ## I used Point-Biserial_Correlation to examine their corollation.
    
    # bin_cols = [col for col in df.columns if "_" in col]
    # correlations = {}
    # for col in bin_cols:
    #     correlation, _ = pointbiserialr(df[col], df['ClaimAmount@Exposure'])
    #     correlations[col] = correlation

    # correlation_df = pd.DataFrame(
    #     list(correlations.items()),
    #     columns=['Transformed_Column', 'Point-Biserial_Correlation']
    # ).sort_values(by="Point-Biserial_Correlation")

    # print(correlation_df)

    ## Used histograph to show what values ClaimAmount@Exposure generally took. Mainly in the 2000 - 3000 range.
    # sns.histplot(df["ClaimAmount@Exposure"], kde=True)
    # plt.show()
    
    return top_correlations.keys()

### Define main Function (LinearRegression() model)

In [14]:
## This is the main function.

## The reason it is defined as a function is because I decided apply the Linear Regression model on smaller subsets.
## Applying it all at once gave bad results, even with some attempts at feature engeineering (more than displayed above).
## It suggests to me that the data is not linearly distributed. By restricting myself to smaller sets, I am hoping to get a more line-like shape.
                                                                ## Basically, every smooth curve "is a line" on a small enough interval.
## On the smaller datasets, I am training the linear model, giving me many different models.
## If the local linear model is worse than the general linear model, the function will choose the general linear model.
## This way, one can optimise the code both globally (genral model), as well as locally.

# The function takes in a dataframe and a list of its features as input.
# The list will come from the previous function correlation_analysis in this script. But one could also generate it in different ways.
def train_linear_model(df, top_correlations):
    
    y = df[["ClaimAmount@Exposure"]]
    ## top_corrlations are the features. It is already made sure not to include the ClaimAmount or its derivatives (our predict).
    ## But here it is checked again, just to make sure, especially if this list comes form an alternative source. 
    X = df[[key for key in top_correlations if key != "ClaimAmount@Exposure" and key != "ClaimAmount"]]

    ## This is important for the loop used below. In case this function is called on empty or one element sets, we can not expect to get a line.
    ## However, I have set this line at 36, since below it, it is unlikely to give us a statistically meaningful result.
    ## Generic solution, bad as it is, is preferable.
    if len(X) < 36:
        return generic_model
    
    # Train and evaluate the model:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Use OLS model for better statistical analysis of the resultant model.
    ## I have not used this as well as I should have!
    ## I am only using the mean_squared_error for automatic evaluation of my model.
    ## The R-squared, p-value etc. were only used to be looked at by me, to test progress of my improvements.
    ## It should be used in the automatic evaluation as well!
    X_train_with_intercept = sm.add_constant(X_train)
    OLS_model = sm.OLS(y_train, X_train_with_intercept).fit()
    
    r_squared = OLS_model.rsquared
    adj_r_squared = OLS_model.rsquared_adj
    f_statistic = OLS_model.fvalue
    f_statistic_prob = OLS_model.f_pvalue
    
    mse = mean_squared_error(y_pred, y_test, squared=False)

    ## Print the extracted statistics. Will be added to the resultant dataset in the main loop, so no need to print them here.
    # print(f"Mean-squared-error: {mse}")
    # print(f"R-squared: {r_squared}")
    # print(f"Adj. R-squared: {adj_r_squared}")
    # print(f"F-statistic: {f_statistic}")
    # print(f"Prob (F-statistic): {f_statistic_prob}")
    
    return [model, mse, r_squared, adj_r_squared, f_statistic, f_statistic_prob]

### Create the generic linear model.

In [15]:
## Here, one should probably try to make more improvements.
## It is used as a fallback, if the specific linear models are not as good, or the dataset they try to fit are too small.
cols_to_check = ["ClaimAmount@Exposure", "BonusMalus"]
df_gen = remove_outliers(df_merged, cols_to_check).dropna()
top_correlations_gen = correlation_analysis(df_gen)
# The name generic_model is used in the funciton train_linear_model. If I change it here, I need to change it there....
generic_model = train_linear_model(df_gen, top_correlations_gen)

### Define sub-datasets for more specific model training.

In [16]:
## I set it up using a generic script that does it. Can also be done using vim and macros to do big lists.
## An entry of the form
## [['Exposure', 0, 0.5], ['VehPower', 0, 5], ['Density', 0, 100]]
## will split df_merged, the main data set which has the "solutions" ClaimAmount@Exposure, as follows: 
## Every entry such that 0 <= Exposure < 0.5, VehPower <= 0 < 5 and 0 <= Density < 100.
## I am keeping strict equality on the right side to avoid double evaluations (it should not cause a problem either way, but still..)

## The idea is to use more sensible numbers, preferably from something like decision trees... Ideally, automatically fed into it.
## Setting an empty list ensures that everything is populated, at least with the generic script. The following will then overwrite.
list_confitions_models = [
    [],
    [['Exposure', 0, 0.5], ['VehPower', 0, 5], ['Density', 0, 100]],
    [['Exposure', 0, 0.5], ['VehPower', 0, 5], ['Density', 100, inf]],
    [['Exposure', 0, 0.5], ['VehPower', 5, inf], ['Density', 0, 100]],
    [['Exposure', 0, 0.5], ['VehPower', 5, inf], ['Density', 100, inf]],
    
    [['Exposure', 0.5, 0.75], ['VehPower', 0, 5], ['Density', 0, 100]],
    [['Exposure', 0.5, 0.75], ['VehPower', 0, 5], ['Density', 100, inf]],
    [['Exposure', 0.5, 0.75], ['VehPower', 5, inf], ['Density', 0, 100]],
    [['Exposure', 0.5, 0.75], ['VehPower', 5, inf], ['Density', 100, inf]],
    
    [['Exposure', 0.75, 0.99], ['VehPower', 0, 5], ['Density', 0, 100]],
    [['Exposure', 0.75, 0.99], ['VehPower', 0, 5], ['Density', 100, inf]],
    [['Exposure', 0.75, 0.99], ['VehPower', 5, inf], ['Density', 0, 100]],
    [['Exposure', 0.75, 0.99], ['VehPower', 5, inf], ['Density', 100, inf]],
    
    [['Exposure', 0.99, 1.01], ['VehPower', 0, 5], ['Density', 0, 100]],
    [['Exposure', 0.99, 1.01], ['VehPower', 0, 5], ['Density', 100, inf]],
    [['Exposure', 0.99, 1.01], ['VehPower', 5, inf], ['Density', 0, 100]],
    [['Exposure', 0.99, 1.01], ['VehPower', 5, inf], ['Density', 100, inf]],

## Some of the entries cause errors. I dealt with a few of them. But at some point, I just commented it out. :)
## This is caused from the entry being too small to be effectively evaluated, hence it's fine to leave it out anyway.
#    [['Exposure', 1.01, inf], ['VehPower', 0, 5], ['Density', 0, 100]],
    [['Exposure', 1.01, inf], ['VehPower', 0, 5], ['Density', 100, inf]],
    [['Exposure', 1.01, inf], ['VehPower', 5, inf], ['Density', 0, 100]],
    [['Exposure', 1.01, inf], ['VehPower', 5, inf], ['Density', 100, inf]]
]

## The main Part

## The initial evaluation; for-loop

In [17]:
for conditions in list_confitions_models:
    ## I errors come up, see where...
    # print(conditions)

    ## Need to set up an inital list of booleans, so that I can store my results after each iteration. Was not able to do it with append...
    combined_mask = np.ones(len(df_merged), dtype=bool)
    for condition in conditions:
        condition_mask = np.logical_and(df_merged[condition[0]] >= condition[1], df_merged[condition[0]] < condition[2])
        combined_mask = np.logical_and(combined_mask, condition_mask)

    ## df_specified becomes our sub-dataset. We will now create a linear model on this, repeating our steps for the generic one.
    df_specified = df_merged[combined_mask]
    cols_to_check = ["ClaimAmount@Exposure", "BonusMalus"]
    df_rem_outliers = remove_outliers(df_specified, cols_to_check).dropna()
    top_correlations = correlation_analysis(df_rem_outliers)
    specific_model = train_linear_model(df_rem_outliers, top_correlations)
    ## Remember, train_linear_model returns not just the model, but also some evaluations. 
    model = specific_model[0]

    ## Apply the same condition to df_rest, so that we can populate the corresponding rows with ClaimAmount@Exposure and the evaluations.
    rest_combined_mask = np.ones(len(df_rest), dtype=bool)
    for condition in conditions:
        rest_condition_mask = np.logical_and(df_rest[condition[0]] >= condition[1], df_rest[condition[0]] < condition[2])
        rest_combined_mask = np.logical_and(rest_combined_mask, rest_condition_mask)

    ## Now we will predict the ClaimAmount@Exposure of the appropriate rows.
    X = df_rest[rest_combined_mask]

    ## If the specific model is worse than the generic one, use generic one.
    ## Here, I am only using mean_squared_error as a metric. This is bad! I should also use R-values etc. for a better evaluation.
    if specific_model[1] > generic_model[1]:
        specific_model = generic_model

    ## Not sure why I have to set model again... but it gave me errors, so I set it by hand it was fixed... I am confused...
    ## if specific_model = generic_model, surely model = specific_model[0] = generic_model[0] ?? Same with correlations...
    if specific_model == generic_model:
        model = generic_model[0]
        top_correlations = top_correlations_gen
    ## If I have zero features or zero entires... nothing to do...
    if len(X[[key for key in top_correlations if key != "ClaimAmount@Exposure" and key != "ClaimAmount"]]) < 1:
        continue
    
    # Predict and add the ClaimAmount@Exposure values (and evaluations) to df_rest.
    predicted_values_spec = model.predict(X[[key for key in top_correlations if key != "ClaimAmount@Exposure" and key != "ClaimAmount"]])
    df_rest.loc[rest_combined_mask, "ClaimAmount@Exposure"] = predicted_values_spec
    df_rest.loc[rest_combined_mask, "MSE"] = specific_model[1]
    df_rest.loc[rest_combined_mask, "R-Squared"] = specific_model[2]
    df_rest.loc[rest_combined_mask, "Adj.R-Squared"] = specific_model[3]
    df_rest.loc[rest_combined_mask, "F-Statistic"] = specific_model[4]
    df_rest.loc[rest_combined_mask, "p-Value"] = specific_model[5]

### Evaluate the final result

In [18]:
## Make sure, everytihng was filled out.
num_missing_values = df_rest["ClaimAmount@Exposure"].size - df_rest["ClaimAmount@Exposure"].dropna().size
print(f"There are << {num_missing_values} >> values missing in the final database")
## Need better evaluation...
average_error = (df_rest["MSE"].sum()) / df_rest["MSE"].size
print(f"The average error is << {average_error: .2f} >>.")
print(f"The error or the generic model is << {generic_model[1]: .2f} >>.")

There are << 0 >> values missing in the final database
The average error is <<  3057.69 >>.
The error or the generic model is <<  4509.41 >>.


In [32]:
df_rest[df_rest["R-Squared"] > 0.3]

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density,Area_B,Area_C,Area_D,...,Region_R91,Region_R93,Region_R94,VehAge@VehPower,ClaimAmount@Exposure,MSE,R-Squared,Adj.R-Squared,F-Statistic,p-Value
